### 0. Import Modules

In [1]:
!pip install featurewiz --ignore-installed --no-deps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 7.8 MB/s 


In [2]:
#!pip install xlrd --ignore-installed --no-deps 

In [3]:
#!pip install Pillow==9.0.0
!pip install category-encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 845 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
featurewiz 0.2.1 requires jupyter, which is not installed.
featurewiz 0.2.1 requires lightgbm>=3.2.1, but you have lightgbm 2.2.3 which is incompatible.
featurewiz 0.2.1 requires Pillow>=9.0.0, but you have pillow 7.1.2 which is incompatible.
featurewiz 0.2.1 requires pyarrow~=7.0.0, but you have pyarrow 6.0.1 which is incompatible.
featurewiz 0.2.1 requires xgboost>=1.5.1, but you have xgboost 0.90 which is incompatible.
featurewiz 0.2.1 requires xlrd>=2.0.0, but you have xlrd 1.1.0 which is incompatible.


In [4]:

from sklearn.linear_model import LogisticRegressionCV, RidgeClassifierCV
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
import numpy as np
import pandas as pd
from numba import njit
import multiprocessing as mp
import timeit
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import datetime as dt

In [6]:
from lightgbm import LGBMClassifier

In [7]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit

In [8]:
from featurewiz import FeatureWiz

Imported 0.2.01 version. Select nrows to a small number when running on huge datasets.
output = featurewiz(dataname, target, corr_limit=0.90, verbose=2, sep=',', 
		header=0, test_data='',feature_engg='', category_encoders='',
		dask_xgboost_flag=False, nrows=None, skip_sulov=False)
Create new features via 'feature_engg' flag : ['interactions','groupby','target']



In [9]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE 

In [10]:
import scipy

### A. Feature Addition Revised

#### Data Download

In [ ]:
X_train_a = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/X_train_1.park")
knowledge = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/knowledge.park")
y_train_a = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/y_train_1.park")

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
df = reduce_mem_usage(X_train_a, verbose=True)

Mem. usage decreased to 555.58 Mb (54.1% reduction)


In [ ]:
df.shape

(1300365, 121)

In [ ]:

#X_train_a["ticker"] = knowledge["ticker"]
#X_train_a["date"] = knowledge["date"]
#X_train_a = X_train_a.sort_values(by=['date'])
#y_train_a = y_train_a.reset_index(drop=True)
#X_train_a = X_train_a.drop(['date'],axis=1)
X_train_a = X_train_a.select_dtypes(include=['float64','int64'])


#X_train_a["ticker"] = knowledge["ticker"]
features = FeatureWiz(corr_limit=0.60, feature_engg='', category_encoders='',dask_xgboost_flag=False, nrows=None, verbose=0)
X_train_a = features.fit_transform(df,y_train_a)



wiz = FeatureWiz(verbose=1)
        X_train_selected = wiz.fit_transform(X_train, y_train)
        X_test_selected = wiz.transform(X_test)
        wiz.features  ### provides a list of selected features ###            
        
############################################################################################
############       F A S T   F E A T U R E  E N G G    A N D    S E L E C T I O N ! ########
# Be judicious with featurewiz. Don't use it to create too many un-interpretable features! #
############################################################################################
correlation limit = 0.6
Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
#### Single_Label Binary_Classification problem ####
    Caution: We will try to reduce the memory usage of dataframe from 565.50 MB
        memory usage after optimization is: 558.06 MB
        decreased by 1.3%
    Loaded train data. Shape

In [ ]:
X_train_a.shape

(1300365, 46)

In [ ]:
del df

In [ ]:
X_train_a["ticker"] = knowledge["ticker"]

#### Feature Generation Functions

In [ ]:


def pk_pk_distance(signal):
    """Computes the peak to peak distance.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which the area under the curve is computed
    Returns
    -------
    float
        peak to peak distance
    """
    return np.abs(np.max(signal) - np.min(signal))
    #return [np.abs(np.max(signal) - np.min(signal))]*len(signal)

def median_abs_diff(signal):
    """Computes median absolute differences of the signal.
   Feature computational cost: 1
   Parameters
   ----------
   signal : nd-array
       Input from which median absolute difference is computed
   Returns
   -------
   float
       Median absolute difference result
   """
    return np.median(np.abs(np.diff(signal)))

def slope(signal):
    """Computes the slope of the signal.
    Slope is computed by fitting a linear equation to the observed data.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which linear equation is computed
    Returns
    -------
    float
        Slope
    """
    t = np.linspace(0, len(signal) - 1, len(signal))
  
    return np.polyfit(t, signal, 1)[0]

def distance(signal):
    """Computes signal traveled distance.
    Calculates the total distance traveled by the signal
    using the hipotenusa between 2 datapoints.
   Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which distance is computed
    Returns
    -------
    float
        Signal distance
    """
    diff_sig = np.diff(signal).astype(float)
    return np.sum([np.sqrt(1 + diff_sig ** 2)])

def zero_cross(signal):
    """Computes Zero-crossing rate of the signal.
    Corresponds to the total number of times that the signal changes from
    positive to negative or vice versa.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which the zero-crossing rate are computed
    Returns
    -------
    int
        Number of times that signal value cross the zero axis
    """
    return len(np.where(np.diff(np.sign(signal)))[0])


def autocorr_norm(signal):
    """Computes the autocorrelation.
    Implementation details and description in:
    https://ccrma.stanford.edu/~orchi/Documents/speaker_recognition_report.pdf
    Parameters
    ----------
    signal : nd-array
        Input from linear prediction coefficients are computed
    Returns
    -------
    nd-array
        Autocorrelation result
    """

    variance = np.var(signal)
    signal = np.copy(signal - signal.mean())
    r = scipy.signal.correlate(signal, signal)[-len(signal):]

    if (signal == 0).all():
        return np.zeros(len(signal))

    acf = r / variance / len(signal)

    return acf

def create_xx(features):
    """Computes the range of features amplitude for the probability density function calculus.
    Parameters
    ----------
    features : nd-array
        Input features
    Returns
    -------
    nd-array
        range of features amplitude
    """

    features_ = np.copy(features)

    if max(features_) < 0:
        max_f = - max(features_)
        min_f = min(features_)
    else:
        min_f = min(features_)
        max_f = max(features_)

    if min(features_) == max(features_):
        xx = np.linspace(min_f, min_f + 10, len(features_))
    else:
        xx = np.linspace(min_f, max_f, len(features_))

    return xx

def kde(features):
    """Computes the probability density function of the input signal using a Gaussian KDE (Kernel Density Estimate)
    Parameters
    ----------
    features : nd-array
        Input from which probability density function is computed
    Returns
    -------
    nd-array
        probability density values
    """
    features_ = np.copy(features)
    xx = create_xx(features_)

    if min(features_) == max(features_):
        noise = np.random.randn(len(features_)) * 0.0001
        features_ = np.copy(features_ + noise)

    kernel = scipy.stats.gaussian_kde(features_, bw_method='silverman')

    return np.array(kernel(xx) / np.sum(kernel(xx)))

def calc_ecdf(signal):
    """Computes the ECDF of the signal.
      Parameters
      ----------
      signal : nd-array
          Input from which ECDF is computed
      Returns
      -------
      nd-array
        Sorted signal and computed ECDF.
      """
    return np.sort(signal), np.arange(1, len(signal)+1)/len(signal)

def calc_centroid(signal, fs):
    """Computes the centroid along the time axis.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which centroid is computed
    fs: int
        Signal sampling frequency
    Returns
    -------
    float
        Temporal centroid
    """

    time = compute_time(signal, fs)

    energy = np.array(signal) ** 2

    t_energy = np.dot(np.array(time), np.array(energy))
    energy_sum = np.sum(energy)

    if energy_sum == 0 or t_energy == 0:
        centroid = 0
    else:
        centroid = t_energy / energy_sum

    return 

def compute_time(signal, fs):
    """Creates the signal correspondent time array.
    Parameters
    ----------
    signal: nd-array
        Input from which the time is computed.
    fs: int
        Sampling Frequency
    Returns
    -------
    time : float list
        Signal time
    """

    return np.arange(0, len(signal))/fs

def gaussian(features):
    """Computes the probability density function of the input signal using a Gaussian function
    Parameters
    ----------
    features : nd-array
        Input from which probability density function is computed
    Returns
    -------
    nd-array
        probability density values
    """

    features_ = np.copy(features)

    xx = create_xx(features_)
    std_value = np.std(features_)
    mean_value = np.mean(features_)

    if std_value == 0:
        return 0.0
    pdf_gauss = scipy.stats.norm.pdf(xx, mean_value, std_value)

    return np.array(pdf_gauss / np.sum(pdf_gauss))


def calc_ecdf(signal):
    """Computes the ECDF of the signal.
      Parameters
      ----------
      signal : nd-array
          Input from which ECDF is computed
      Returns
      -------
      nd-array
        Sorted signal and computed ECDF.
      """
    return np.sort(signal), np.arange(1, len(signal)+1)/len(signal)

def positive_turning(signal):
    """Computes number of positive turning points of the signal.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which  positive turning points are counted
    Returns
    -------
    float
        Number of positive turning points
    """
    diff_sig = np.diff(signal)

    array_signal = np.arange(len(diff_sig[:-1]))

    positive_turning_pts = np.where((diff_sig[array_signal + 1] < 0) & (diff_sig[array_signal] > 0))[0]

    return len(positive_turning_pts)

def abs_energy(signal):
    """Computes the absolute energy of the signal.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which the area under the curve is computed
    Returns
    -------
    float
        Absolute energy
    """
    return np.sum(np.abs(signal) ** 2)

def skewness(signal):
    """Computes skewness of the signal.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which skewness is computed
    Returns
    -------
    int
        Skewness result
    """
    return scipy.stats.skew(signal)

def kurtosis(signal):
    """Computes kurtosis of the signal.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which kurtosis is computed
    Returns
    -------
    float
        Kurtosis result
    """
    return scipy.stats.kurtosis(signal)
def negative_turning(signal):
    """Computes number of negative turning points of the signal.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which minimum number of negative turning points are counted
    Returns
    -------
    float
        Number of negative turning points
    """
    diff_sig = np.diff(signal)
    array_signal = np.arange(len(diff_sig[:-1]))
    negative_turning_pts = np.where((diff_sig[array_signal] < 0) & (diff_sig[array_signal + 1] > 0))[0]

    return len(negative_turning_pts)

def mean_diff(signal):
    """Computes mean of differences of the signal.
   Feature computational cost: 1
   Parameters
   ----------
   signal : nd-array
       Input from which mean of differences is computed
   Returns
   -------
   float
       Mean difference result
   """
    return np.mean(np.diff(signal))

def median_abs_diff(signal):
    """Computes median absolute differences of the signal.
   Feature computational cost: 1
   Parameters
   ----------
   signal : nd-array
       Input from which median absolute difference is computed
   Returns
   -------
   float
       Median absolute difference result
   """
    return np.median(np.abs(np.diff(signal)))

def median_diff(signal):
    """Computes median of differences of the signal.
   Feature computational cost: 1
   Parameters
   ----------
   signal : nd-array
       Input from which median of differences is computed
   Returns
   -------
   float
       Median difference result
   """
    return np.median(np.diff(signal))

def total_energy(signal, fs):
    """Computes the total energy of the signal.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Signal from which total energy is computed
    fs : int
        Sampling frequency
    Returns
    -------
    float
        Total energy
    """
    time = compute_time(signal, fs)

    return np.sum(np.array(signal) ** 2) / (time[-1] - time[0])

def distance(signal):
    """Computes signal traveled distance.
    Calculates the total distance traveled by the signal
    using the hipotenusa between 2 datapoints.
   Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which distance is computed
    Returns
    -------
    float
        Signal distance
    """
    diff_sig = np.diff(signal).astype(float)
    return np.sum([np.sqrt(1 + diff_sig ** 2)])

def auc(signal, fs):
    """Computes the area under the curve of the signal computed with trapezoid rule.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which the area under the curve is computed
    fs : int
        Sampling Frequency
    Returns
    -------
    float
        The area under the curve value
    """
    t = compute_time(signal, fs)

    return np.sum(0.5 * np.diff(t) * np.abs(np.array(signal[:-1]) + np.array(signal[1:])))

### Numba not supported :
#  skewness, kurtosis, distance, 

###Not working:
# calc_centroid, 


#### New Features

In [ ]:
from scipy import signal

In [ ]:
def variance_larger_than_standard_deviation(x):
    """
    Is variance higher than the standard deviation?
    Boolean variable denoting if the variance of x is greater than its standard deviation. Is equal to variance of x
    being larger than 1
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: bool
    """
    y = np.var(x)
    return y > np.sqrt(y)

def sum_values(x):
    """
    Calculates the sum over the time series values
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    if len(x) == 0:
        return 0

    return np.sum(x)

def cid_ce(x, normalize=False):
    """
    This function calculator is an estimate for a time series complexity [1] (A more complex time series has more peaks,
    valleys etc.). It calculates the value of
    .. math::
        \\sqrt{ \\sum_{i=1}^{n-1} ( x_{i} - x_{i-1})^2 }
    .. rubric:: References
    |  [1] Batista, Gustavo EAPA, et al (2014).
    |  CID: an efficient complexity-invariant distance for time series.
    |  Data Mining and Knowledge Discovery 28.3 (2014): 634-669.
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :param normalize: should the time series be z-transformed?
    :type normalize: bool
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    if normalize:
        s = np.std(x)
        if s != 0:
            x = (x - np.mean(x)) / s
        else:
            return 0.0

    x = np.diff(x)
    return np.sqrt(np.dot(x, x))

def mean_second_derivative_central(x):
    """
    Returns the mean value of a central approximation of the second derivative
    .. math::
        \\frac{1}{2(n-2)} \\sum_{i=1,\\ldots, n-1}  \\frac{1}{2} (x_{i+2} - 2 \\cdot x_{i+1} + x_i)
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    x = np.asarray(x)
    return (x[-1] - x[-2] - x[1] + x[0]) / (2 * (len(x) - 2)) if len(x) > 2 else np.NaN

def standard_deviation(x):
    """
    Returns the standard deviation of x
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    return np.std(x)

def variance(x):
    """
    Returns the variance of x
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    return np.var(x)
def skewness(x):
    """
    Returns the sample skewness of x (calculated with the adjusted Fisher-Pearson standardized
    moment coefficient G1).
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, pd.Series):
        x = pd.Series(x)
    return pd.Series.skew(x)

def kurtosis(x):
    """
    Returns the kurtosis of x (calculated with the adjusted Fisher-Pearson standardized
    moment coefficient G2).
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, pd.Series):
        x = pd.Series(x)
    return pd.Series.kurtosis(x)

def count_above_mean(x):
    """
    Returns the number of values in x that are higher than the mean of x
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    m = np.mean(x)
    return np.where(x > m)[0].size

def count_below_mean(x):
    """
    Returns the number of values in x that are lower than the mean of x
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    m = np.mean(x)
    return np.where(x < m)[0].size

def percentage_of_reoccurring_values_to_all_values(x):
    """
    Returns the percentage of values that are present in the time series
    more than once.
        len(different values occurring more than once) / len(different values)
    This means the percentage is normalized to the number of unique values,
    in contrast to the percentage_of_reoccurring_datapoints_to_all_datapoints.
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    if len(x) == 0:
        return np.nan

    unique, counts = np.unique(x, return_counts=True)

    if counts.shape[0] == 0:
        return 0

    return np.sum(counts > 1) / float(counts.shape[0])

def ratio_value_number_to_time_series_length(x):
    """
    Returns a factor which is 1 if all values in the time series occur only once,
    and below one if this is not the case.
    In principle, it just returns
        # unique values / # values
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    if x.size == 0:
        return np.nan

    return np.unique(x).size / x.size


In [ ]:
### Not working: fft_mean_coeff

In [ ]:
###https://github.com/blue-yonder/tsfresh/blob/main/tsfresh/feature_extraction/feature_calculators.py

#### Store Features

In [ ]:
y_train_a = y_train_a.reset_index(drop=True)

In [ ]:
#functs = ['zero_cross','mean','abs_energy','pk_pk_distance','mean_diff']
#functions = [zero_cross,np.mean,abs_energy,pk_pk_distance,mean_diff]
#functs =    ['count_above_mean','ratio_value_number_to_time_series_length','mean_second_derivative_central']
#functions = [count_above_mean,ratio_value_number_to_time_series_length,mean_second_derivative_central]
functs = ['variance_larger_than_standard_deviation','zero_cross','mean','abs_energy','pk_pk_distance','mean_diff']
functions = [variance_larger_than_standard_deviation,zero_cross,np.mean,abs_energy,pk_pk_distance,mean_diff]


res = []
windows = [4]
for window in windows:
  for i in range(len(functs)):
    try:
      X = X_train_a.groupby('ticker').rolling(window).apply(functions[i],engine='numba',raw=True).add_suffix("{}".format(functs[i]+str(window)))
    except:
      X = X_train_a.groupby('ticker').rolling(window).apply(functions[i]).add_suffix("{}".format(functs[i]))
    X = X.reset_index(drop=True)
    features = FeatureWiz(corr_limit=0.30, feature_engg='', category_encoders='',dask_xgboost_flag=False, nrows=None, verbose=0)
    X = features.fit_transform(X,y_train_a)
    res.append(X)

wiz = FeatureWiz(verbose=1)
        X_train_selected = wiz.fit_transform(X_train, y_train)
        X_test_selected = wiz.transform(X_test)
        wiz.features  ### provides a list of selected features ###            
        
############################################################################################
############       F A S T   F E A T U R E  E N G G    A N D    S E L E C T I O N ! ########
# Be judicious with featurewiz. Don't use it to create too many un-interpretable features! #
############################################################################################
correlation limit = 0.3
Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
#### Single_Label Binary_Classification problem ####
    Loaded train data. Shape = (1300365, 47)
#### Single_Label Binary_Classification problem ####
No test data filename given...
Classifying features using a random sample of 10000 rows fr

#### Append the results

In [ ]:
res.append(X_train_a)

In [ ]:
from functools import reduce
df = reduce(lambda x, y: x.join(y), res)

In [ ]:
del res
del X_train_a

In [ ]:
#df = df2.copy()
#del df2

In [ ]:
df.shape

(1300365, 72)

In [ ]:
### Feature Selection at each stage to reduce data size
features = FeatureWiz(corr_limit=0.60, feature_engg='', category_encoders='',dask_xgboost_flag=False, nrows=None, verbose=0)
df = features.fit_transform(df,y_train_a)


wiz = FeatureWiz(verbose=1)
        X_train_selected = wiz.fit_transform(X_train, y_train)
        X_test_selected = wiz.transform(X_test)
        wiz.features  ### provides a list of selected features ###            
        
############################################################################################
############       F A S T   F E A T U R E  E N G G    A N D    S E L E C T I O N ! ########
# Be judicious with featurewiz. Don't use it to create too many un-interpretable features! #
############################################################################################
correlation limit = 0.6
Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
#### Single_Label Binary_Classification problem ####
    Caution: We will try to reduce the memory usage of dataframe from 488.62 MB
        memory usage after optimization is: 564.26 MB
        decreased by -15.5%
    Loaded train data. Sha

In [ ]:
df.shape

(1300365, 19)

#### Training Preparations

In [ ]:
#df2 = df.copy()

In [ ]:
y_train_a = y_train_a.reset_index(drop=True)

In [ ]:
df['target'] = y_train_a['target']


In [ ]:
df = df.dropna(axis=0)
df = df.reset_index(drop=True)

In [ ]:
y = df['target']
df = df.drop(['target'],axis=1)


In [ ]:
df.shape

(1266867, 19)

In [ ]:
#del df2
df2 = reduce_mem_usage(df, verbose=True)

Mem. usage decreased to 45.91 Mb (75.0% reduction)


In [ ]:
df2.shape

(1266867, 19)

In [ ]:
del df

In [ ]:
#sm = SMOTE(random_state=42)
#df, y = sm.fit_resample(df, y)
df_train, df_test, y_train, y_test = train_test_split(df2, y, test_size=0.33, shuffle=False, random_state=42)

In [ ]:
###Model training and testing
#clf = LGBMClassifier()
#clf.fit(df_train, y_train)

In [ ]:
###https://www.kaggle.com/code/mlisovyi/lighgbm-hyperoptimisation-with-f1-macro/notebook
clf = LGBMClassifier()
clf.fit(df_train, y_train)

LGBMClassifier()

####Evaluations

In [ ]:
### Perform the prediction and compute the scores
ypred = clf.predict(df_test)
accuracyscore = accuracy_score(ypred,y_test)
f1score = f1_score(ypred,y_test)
cnfscore = confusion_matrix(ypred,y_test)
roc_score = roc_auc_score(ypred,y_test)

In [ ]:
### Print the Results
print("Accuracy ",accuracyscore)
print("F1 Score ",f1score)
print(cnfscore)
print("Baseline roc_auc score {:.3f}".format(roc_score))

Accuracy  0.9794697979032069
F1 Score  0.0
[[409484   8577]
 [     6      0]]
Baseline roc_auc score 0.490


In [ ]:
##### Smote predictions
#from imblearn.over_sampling import ADASYN 
#ada = ADASYN(random_state=42)
#df_train_res, y_train_res = ada.fit_resample(df_train, y_train)
# clf2 = LGBMClassifier()
# clf2.fit(df_train_res, y_train_res)
# ypred = clf2.predict(df_test)
# accuracyscore = accuracy_score(ypred,y_test)
# f1score = f1_score(ypred,y_test)
# cnfscore = confusion_matrix(ypred,y_test)
# roc_score = roc_auc_score(ypred,y_test)
# ### Print the Results
# print("Accuracy ",accuracyscore)
# print("F1 Score ",f1score)
# print(cnfscore)
# print("Baseline roc_auc score {:.3f}".format(roc_score))

**Performance Scores**
1. Baseline +np.mean (4,8,12) = 0.544
2. Baseline +np.mean(4,8,12) + Adasyn = 0.546

### B. Feature Addition - 2 -:Active

#### Data Download

In [11]:
X_train_a = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/X_train_1.park")
knowledge = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/knowledge.park")
y_train_a = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/y_train_1.park")

In [12]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [13]:
df = reduce_mem_usage(X_train_a, verbose=True)

Mem. usage decreased to 555.58 Mb (54.1% reduction)


In [14]:
df.shape

(1300365, 121)

In [15]:

#X_train_a["ticker"] = knowledge["ticker"]
#X_train_a["date"] = knowledge["date"]
#X_train_a = X_train_a.sort_values(by=['date'])
#y_train_a = y_train_a.reset_index(drop=True)
#X_train_a = X_train_a.drop(['date'],axis=1)
X_train_a = X_train_a.select_dtypes(include=['float64','int64'])


#X_train_a["ticker"] = knowledge["ticker"]
features = FeatureWiz(corr_limit=0.60, feature_engg='', category_encoders='',dask_xgboost_flag=False, nrows=None, verbose=0)
X_train_a = features.fit_transform(df,y_train_a)



wiz = FeatureWiz(verbose=1)
        X_train_selected = wiz.fit_transform(X_train, y_train)
        X_test_selected = wiz.transform(X_test)
        wiz.features  ### provides a list of selected features ###            
        
############################################################################################
############       F A S T   F E A T U R E  E N G G    A N D    S E L E C T I O N ! ########
# Be judicious with featurewiz. Don't use it to create too many un-interpretable features! #
############################################################################################
correlation limit = 0.6
Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
#### Single_Label Binary_Classification problem ####
    Caution: We will try to reduce the memory usage of dataframe from 565.50 MB
        memory usage after optimization is: 558.06 MB
        decreased by 1.3%
    Loaded train data. Shape

In [16]:
X_train_a.shape

(1300365, 46)

In [17]:
del df

In [18]:
X_train_a["ticker"] = knowledge["ticker"]

#### Feature Generation Functions

In [19]:

def pk_pk_distance(signal):
    """Computes the peak to peak distance.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which the area under the curve is computed
    Returns
    -------
    float
        peak to peak distance
    """
    return np.abs(np.max(signal) - np.min(signal))
    #return [np.abs(np.max(signal) - np.min(signal))]*len(signal)

def median_abs_diff(signal):
    """Computes median absolute differences of the signal.
   Feature computational cost: 1
   Parameters
   ----------
   signal : nd-array
       Input from which median absolute difference is computed
   Returns
   -------
   float
       Median absolute difference result
   """
    return np.median(np.abs(np.diff(signal)))

def slope(signal):
    """Computes the slope of the signal.
    Slope is computed by fitting a linear equation to the observed data.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which linear equation is computed
    Returns
    -------
    float
        Slope
    """
    t = np.linspace(0, len(signal) - 1, len(signal))
  
    return np.polyfit(t, signal, 1)[0]

def distance(signal):
    """Computes signal traveled distance.
    Calculates the total distance traveled by the signal
    using the hipotenusa between 2 datapoints.
   Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which distance is computed
    Returns
    -------
    float
        Signal distance
    """
    diff_sig = np.diff(signal).astype(float)
    return np.sum([np.sqrt(1 + diff_sig ** 2)])

def zero_cross(signal):
    """Computes Zero-crossing rate of the signal.
    Corresponds to the total number of times that the signal changes from
    positive to negative or vice versa.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which the zero-crossing rate are computed
    Returns
    -------
    int
        Number of times that signal value cross the zero axis
    """
    return len(np.where(np.diff(np.sign(signal)))[0])


def autocorr_norm(signal):
    """Computes the autocorrelation.
    Implementation details and description in:
    https://ccrma.stanford.edu/~orchi/Documents/speaker_recognition_report.pdf
    Parameters
    ----------
    signal : nd-array
        Input from linear prediction coefficients are computed
    Returns
    -------
    nd-array
        Autocorrelation result
    """

    variance = np.var(signal)
    signal = np.copy(signal - signal.mean())
    r = scipy.signal.correlate(signal, signal)[-len(signal):]

    if (signal == 0).all():
        return np.zeros(len(signal))

    acf = r / variance / len(signal)

    return acf

def create_xx(features):
    """Computes the range of features amplitude for the probability density function calculus.
    Parameters
    ----------
    features : nd-array
        Input features
    Returns
    -------
    nd-array
        range of features amplitude
    """

    features_ = np.copy(features)

    if max(features_) < 0:
        max_f = - max(features_)
        min_f = min(features_)
    else:
        min_f = min(features_)
        max_f = max(features_)

    if min(features_) == max(features_):
        xx = np.linspace(min_f, min_f + 10, len(features_))
    else:
        xx = np.linspace(min_f, max_f, len(features_))

    return xx

def kde(features):
    """Computes the probability density function of the input signal using a Gaussian KDE (Kernel Density Estimate)
    Parameters
    ----------
    features : nd-array
        Input from which probability density function is computed
    Returns
    -------
    nd-array
        probability density values
    """
    features_ = np.copy(features)
    xx = create_xx(features_)

    if min(features_) == max(features_):
        noise = np.random.randn(len(features_)) * 0.0001
        features_ = np.copy(features_ + noise)

    kernel = scipy.stats.gaussian_kde(features_, bw_method='silverman')

    return np.array(kernel(xx) / np.sum(kernel(xx)))

def calc_ecdf(signal):
    """Computes the ECDF of the signal.
      Parameters
      ----------
      signal : nd-array
          Input from which ECDF is computed
      Returns
      -------
      nd-array
        Sorted signal and computed ECDF.
      """
    return np.sort(signal), np.arange(1, len(signal)+1)/len(signal)

def calc_centroid(signal, fs):
    """Computes the centroid along the time axis.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which centroid is computed
    fs: int
        Signal sampling frequency
    Returns
    -------
    float
        Temporal centroid
    """

    time = compute_time(signal, fs)

    energy = np.array(signal) ** 2

    t_energy = np.dot(np.array(time), np.array(energy))
    energy_sum = np.sum(energy)

    if energy_sum == 0 or t_energy == 0:
        centroid = 0
    else:
        centroid = t_energy / energy_sum

    return 

def compute_time(signal, fs):
    """Creates the signal correspondent time array.
    Parameters
    ----------
    signal: nd-array
        Input from which the time is computed.
    fs: int
        Sampling Frequency
    Returns
    -------
    time : float list
        Signal time
    """

    return np.arange(0, len(signal))/fs

def gaussian(features):
    """Computes the probability density function of the input signal using a Gaussian function
    Parameters
    ----------
    features : nd-array
        Input from which probability density function is computed
    Returns
    -------
    nd-array
        probability density values
    """

    features_ = np.copy(features)

    xx = create_xx(features_)
    std_value = np.std(features_)
    mean_value = np.mean(features_)

    if std_value == 0:
        return 0.0
    pdf_gauss = scipy.stats.norm.pdf(xx, mean_value, std_value)

    return np.array(pdf_gauss / np.sum(pdf_gauss))


def calc_ecdf(signal):
    """Computes the ECDF of the signal.
      Parameters
      ----------
      signal : nd-array
          Input from which ECDF is computed
      Returns
      -------
      nd-array
        Sorted signal and computed ECDF.
      """
    return np.sort(signal), np.arange(1, len(signal)+1)/len(signal)

def positive_turning(signal):
    """Computes number of positive turning points of the signal.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which  positive turning points are counted
    Returns
    -------
    float
        Number of positive turning points
    """
    diff_sig = np.diff(signal)

    array_signal = np.arange(len(diff_sig[:-1]))

    positive_turning_pts = np.where((diff_sig[array_signal + 1] < 0) & (diff_sig[array_signal] > 0))[0]

    return len(positive_turning_pts)

def abs_energy(signal):
    """Computes the absolute energy of the signal.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which the area under the curve is computed
    Returns
    -------
    float
        Absolute energy
    """
    return np.sum(np.abs(signal) ** 2)

def skewness(signal):
    """Computes skewness of the signal.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which skewness is computed
    Returns
    -------
    int
        Skewness result
    """
    return scipy.stats.skew(signal)

def kurtosis(signal):
    """Computes kurtosis of the signal.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which kurtosis is computed
    Returns
    -------
    float
        Kurtosis result
    """
    return scipy.stats.kurtosis(signal)
def negative_turning(signal):
    """Computes number of negative turning points of the signal.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which minimum number of negative turning points are counted
    Returns
    -------
    float
        Number of negative turning points
    """
    diff_sig = np.diff(signal)
    array_signal = np.arange(len(diff_sig[:-1]))
    negative_turning_pts = np.where((diff_sig[array_signal] < 0) & (diff_sig[array_signal + 1] > 0))[0]

    return len(negative_turning_pts)

def mean_diff(signal):
    """Computes mean of differences of the signal.
   Feature computational cost: 1
   Parameters
   ----------
   signal : nd-array
       Input from which mean of differences is computed
   Returns
   -------
   float
       Mean difference result
   """
    return np.mean(np.diff(signal))

def median_abs_diff(signal):
    """Computes median absolute differences of the signal.
   Feature computational cost: 1
   Parameters
   ----------
   signal : nd-array
       Input from which median absolute difference is computed
   Returns
   -------
   float
       Median absolute difference result
   """
    return np.median(np.abs(np.diff(signal)))

def median_diff(signal):
    """Computes median of differences of the signal.
   Feature computational cost: 1
   Parameters
   ----------
   signal : nd-array
       Input from which median of differences is computed
   Returns
   -------
   float
       Median difference result
   """
    return np.median(np.diff(signal))

def total_energy(signal, fs):
    """Computes the total energy of the signal.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Signal from which total energy is computed
    fs : int
        Sampling frequency
    Returns
    -------
    float
        Total energy
    """
    time = compute_time(signal, fs)

    return np.sum(np.array(signal) ** 2) / (time[-1] - time[0])

def distance(signal):
    """Computes signal traveled distance.
    Calculates the total distance traveled by the signal
    using the hipotenusa between 2 datapoints.
   Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which distance is computed
    Returns
    -------
    float
        Signal distance
    """
    diff_sig = np.diff(signal).astype(float)
    return np.sum([np.sqrt(1 + diff_sig ** 2)])

def auc(signal, fs):
    """Computes the area under the curve of the signal computed with trapezoid rule.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which the area under the curve is computed
    fs : int
        Sampling Frequency
    Returns
    -------
    float
        The area under the curve value
    """
    t = compute_time(signal, fs)

    return np.sum(0.5 * np.diff(t) * np.abs(np.array(signal[:-1]) + np.array(signal[1:])))

### Numba not supported :
#  skewness, kurtosis, distance, 

###Not working:
# calc_centroid, 


#### New Features

In [20]:
from scipy import signal

In [21]:
def variance_larger_than_standard_deviation(x):
    """
    Is variance higher than the standard deviation?
    Boolean variable denoting if the variance of x is greater than its standard deviation. Is equal to variance of x
    being larger than 1
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: bool
    """
    y = np.var(x)
    return y > np.sqrt(y)

def sum_values(x):
    """
    Calculates the sum over the time series values
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    if len(x) == 0:
        return 0

    return np.sum(x)

def cid_ce(x, normalize=False):
    """
    This function calculator is an estimate for a time series complexity [1] (A more complex time series has more peaks,
    valleys etc.). It calculates the value of
    .. math::
        \\sqrt{ \\sum_{i=1}^{n-1} ( x_{i} - x_{i-1})^2 }
    .. rubric:: References
    |  [1] Batista, Gustavo EAPA, et al (2014).
    |  CID: an efficient complexity-invariant distance for time series.
    |  Data Mining and Knowledge Discovery 28.3 (2014): 634-669.
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :param normalize: should the time series be z-transformed?
    :type normalize: bool
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    if normalize:
        s = np.std(x)
        if s != 0:
            x = (x - np.mean(x)) / s
        else:
            return 0.0

    x = np.diff(x)
    return np.sqrt(np.dot(x, x))

def mean_second_derivative_central(x):
    """
    Returns the mean value of a central approximation of the second derivative
    .. math::
        \\frac{1}{2(n-2)} \\sum_{i=1,\\ldots, n-1}  \\frac{1}{2} (x_{i+2} - 2 \\cdot x_{i+1} + x_i)
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    x = np.asarray(x)
    return (x[-1] - x[-2] - x[1] + x[0]) / (2 * (len(x) - 2)) if len(x) > 2 else np.NaN

def standard_deviation(x):
    """
    Returns the standard deviation of x
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    return np.std(x)

def variance(x):
    """
    Returns the variance of x
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    return np.var(x)
def skewness(x):
    """
    Returns the sample skewness of x (calculated with the adjusted Fisher-Pearson standardized
    moment coefficient G1).
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, pd.Series):
        x = pd.Series(x)
    return pd.Series.skew(x)

def kurtosis(x):
    """
    Returns the kurtosis of x (calculated with the adjusted Fisher-Pearson standardized
    moment coefficient G2).
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, pd.Series):
        x = pd.Series(x)
    return pd.Series.kurtosis(x)

def count_above_mean(x):
    """
    Returns the number of values in x that are higher than the mean of x
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    m = np.mean(x)
    return np.where(x > m)[0].size

def count_below_mean(x):
    """
    Returns the number of values in x that are lower than the mean of x
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    m = np.mean(x)
    return np.where(x < m)[0].size

def percentage_of_reoccurring_values_to_all_values(x):
    """
    Returns the percentage of values that are present in the time series
    more than once.
        len(different values occurring more than once) / len(different values)
    This means the percentage is normalized to the number of unique values,
    in contrast to the percentage_of_reoccurring_datapoints_to_all_datapoints.
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    if len(x) == 0:
        return np.nan

    unique, counts = np.unique(x, return_counts=True)

    if counts.shape[0] == 0:
        return 0

    return np.sum(counts > 1) / float(counts.shape[0])

def ratio_value_number_to_time_series_length(x):
    """
    Returns a factor which is 1 if all values in the time series occur only once,
    and below one if this is not the case.
    In principle, it just returns
        # unique values / # values
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    if x.size == 0:
        return np.nan

    return np.unique(x).size / x.size

def ratio_beyond_r_sigma(x, r=1.61):
    """
    Ratio of values that are more than r * std(x) (so r times sigma) away from the mean of x.
    :param x: the time series to calculate the feature of
    :type x: iterable
    :param r: the ratio to compare with
    :type r: float
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.sum(np.abs(x - np.mean(x)) > r * np.std(x)) / x.size

def has_duplicate_min(x):
    """
    Checks if the minimal value of x is observed more than once
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: bool
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.sum(x == np.min(x)) >= 2

def first_location_of_maximum(x):
    """
    Returns the first location of the maximum value of x.
    The position is calculated relatively to the length of x.
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.argmax(x) / len(x) if len(x) > 0 else np.NaN

def sum_of_reoccurring_values(x):
    """
    Returns the sum of all values, that are present in the time series
    more than once.
    For example
        sum_of_reoccurring_values([2, 2, 2, 2, 1]) = 2
    as 2 is a reoccurring value, so it is summed up with all
    other reoccuring values (there is none), so the result is 2.
    This is in contrast to ``sum_of_reoccurring_data_points``,
    where each reoccuring value is only counted as often as
    it is present in the data.
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    unique, counts = np.unique(x, return_counts=True)
    counts[counts < 2] = 0
    counts[counts > 1] = 1
    return np.sum(counts * unique)

def binned_entropy(x, max_bins=3):
    """
    First bins the values of x into max_bins equidistant bins.
    Then calculates the value of
    .. math::
        - \\sum_{k=0}^{min(max\\_bins, len(x))} p_k log(p_k) \\cdot \\mathbf{1}_{(p_k > 0)}
    where :math:`p_k` is the percentage of samples in bin :math:`k`.
    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :param max_bins: the maximal number of bins
    :type max_bins: int
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)

    # nan makes no sense here
    if np.isnan(x).any():
        return np.nan

    hist, bin_edges = np.histogram(x, bins=max_bins)
    probs = hist / x.size
    probs[probs == 0] = 1.0
    return -np.sum(probs * np.log(probs))

In [22]:
### Not working: fft_mean_coeff

In [23]:
###https://github.com/blue-yonder/tsfresh/blob/main/tsfresh/feature_extraction/feature_calculators.py

#### Store Features

In [24]:
y_train_a = y_train_a.reset_index(drop=True)

In [25]:
#functs = ['zero_cross','mean','abs_energy','pk_pk_distance','mean_diff','mean_second_derivative_central','ratio_value_number_to_time_series_length','count_above_mean']
#functions = [zero_cross,np.mean,abs_energy,pk_pk_distance,mean_diff,mean_second_derivative_central,ratio_value_number_to_time_series_length,count_above_mean]
#functs =    ['count_above_mean','ratio_value_number_to_time_series_length','mean_second_derivative_central']
#functions = [count_above_mean,ratio_value_number_to_time_series_length,mean_second_derivative_central]
#functs = ['variance_larger_than_standard_deviation','zero_cross','mean','abs_energy','pk_pk_distance','mean_diff']
#functions = [variance_larger_than_standard_deviation,zero_cross,np.mean,abs_energy,pk_pk_distance,mean_diff]
functs =    ['binned_entropy','sum_of_reoccurring_values']
functions = [binned_entropy,sum_of_reoccurring_values]

res = []
windows = [8]
for window in windows:
  for i in range(len(functs)):
    try:
      X = X_train_a.groupby('ticker').rolling(window).apply(functions[i],engine='numba',raw=True).add_suffix("{}".format(functs[i]+str(window)))
    except:
      X = X_train_a.groupby('ticker').rolling(window).apply(functions[i]).add_suffix("{}".format(functs[i]))
    X = X.reset_index(drop=True)
    features = FeatureWiz(corr_limit=0.30, feature_engg='', category_encoders='',dask_xgboost_flag=False, nrows=None, verbose=0)
    X = features.fit_transform(X,y_train_a)
    res.append(X)

KeyboardInterrupt: ignored

#### Append the results

In [ ]:
res.append(X_train_a)

In [ ]:
from functools import reduce
df = reduce(lambda x, y: x.join(y), res)

In [ ]:
del res
del X_train_a

In [ ]:
#df = df2.copy()
#del df2

In [ ]:
df.shape

In [ ]:
### Feature Selection at each stage to reduce data size
features = FeatureWiz(corr_limit=0.60, feature_engg='', category_encoders='',dask_xgboost_flag=False, nrows=None, verbose=0)
df = features.fit_transform(df,y_train_a)


In [ ]:
df.shape

#### Training Preparations

In [ ]:
y_train_a = y_train_a.reset_index(drop=True)

In [ ]:
df['target'] = y_train_a['target']


In [ ]:
df.shape

In [ ]:
knowledge = knowledge.reset_index(drop=True)

In [ ]:
df['date'] = knowledge['date']

In [ ]:
df['ticker'] = knowledge['ticker']

In [ ]:
#df.to_csv('feature_trained.csv')

In [ ]:
#y = df['target']
#df = df.drop(['target'],axis=1)

In [ ]:
# df = df.dropna(axis=0)
# df = df.reset_index(drop=True)

In [ ]:
#features = df.columns
#features = features.drop(['date', 'target','ticker'])

In [ ]:
# def time_series_split(data, split_percent = 0.7):
#   min_date = data.date.min()
#   max_date = data.date.max()
#   #print("Min:", min_date, "Max:", max_date)
#   time_between = max_date - min_date
#   train_cutoff = min_date + split_percent*time_between
#   #print("train_cutoff:", train_cutoff)
#   train_df = data[data.date <= train_cutoff]['formulae_index'].tolist()
#   test_df = data[data.date > train_cutoff]['formulae_index'].index.tolist()
#   #print("Train:", train_df.date.min(), train_df.date.max(), train_df.shape)
#   #print("Test:", test_df.date.min(), test_df.date.max(), test_df.shape)
#   return train_df, test_df, train_cutoff 

In [ ]:
# train_final = []
# test_final = []
# X_train_a['formulae_index'] = X_train_a.index
# tickers = X_train_a['ticker'].unique().tolist()
# for t in tickers:
#   df_temp = X_train_a[X_train_a['ticker'] == t]
#   train_data, test_data, train_cutoff = time_series_split(df_temp, split_percent = 0.8)
#   if ((len(train_data)>0)&(len(test_data)>0)):
#     train_final.extend(train_data)
#     test_final.extend(test_data)
#   del df_temp


In [ ]:
#train_complete.drop(labels=['date','ticker','index'], axis = 1, inplace = True)
#test_complete.drop(labels=['date','ticker','index'], axis = 1, inplace = True)
#train_complete.shape

In [ ]:
#test_complete.shape

#### Reserved Indexes Put

In [ ]:
import pickle

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
with open("train_final", "rb") as fp:   # Unpickling
  train_final = pickle.load(fp)

with open("test_final", "rb") as fp:   # Unpickling
  test_final = pickle.load(fp)

In [ ]:
#df['formulae_index'] = df.index

In [ ]:
test_complete = df[df.index.isin(test_final)]
train_complete = df[df.index.isin(train_final)]

In [ ]:
# df = df.reset_index(drop=True)
# df = df.dropna(axis=0)
# df = df.reset_index(drop=True)

In [ ]:
# test_complete = df[df.formulae_index.isin(test_final)]
# train_complete = df[df.formulae_index.isin(train_final)]

In [ ]:
train_complete = train_complete.reset_index(drop=True)
train_complete = train_complete.dropna(axis=0)
train_complete = train_complete.reset_index(drop=True)

In [ ]:
test_complete = test_complete.reset_index(drop=True)
test_complete = test_complete.dropna(axis=0)
test_complete = test_complete.reset_index(drop=True)

In [ ]:
y_train = train_complete['target']

In [ ]:
y_test = test_complete['target']

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
train_complete.shape

In [ ]:
#sm = SMOTE(random_state=42)
#df, y = sm.fit_resample(df, y)
#df_train, df_test, y_train, y_test = train_test_split(df, y, test_size=0.33, shuffle=True, random_state=42)

In [ ]:
# try:
#   train_complete = train_complete.drop(labels=['date','ticker','index','formulae_index'], axis = 1, inplace = True)
#   test_complete = test_complete.drop(labels=['date','ticker','index','formulae_index'], axis = 1, inplace = True)
# except:
train_complete = train_complete.drop(labels=['date','ticker','target'], axis = 1)
test_complete = test_complete.drop(labels=['date','ticker','target'], axis = 1)

In [ ]:
train_complete.shape

In [ ]:
test_complete.shape

#### Evaluations

In [ ]:
###Model training and testing
clf = LGBMClassifier()
clf.fit(train_complete, y_train)

In [ ]:
### Perform the prediction and compute the scores
ypred = clf.predict(test_complete)
accuracyscore = accuracy_score(ypred,y_test)
f1score = f1_score(ypred,y_test)
cnfscore = confusion_matrix(ypred,y_test)
roc_score = roc_auc_score(ypred,y_test)

In [ ]:
### Print the Results
print("Accuracy ",accuracyscore)
print("F1 Score ",f1score)
print(cnfscore)
print("Baseline roc_auc score {:.3f}".format(roc_score))

In [ ]:
### Baseline roc_auc score 0.
### All feature roc_auc_Score 0.530
### With SULOV 0.554

In [ ]:
pd.DataFrame(y_train).value_counts()

In [ ]:
pd.DataFrame(y_test).value_counts()

**Performance Results**
1. Baseline Score: 0.530
2. lr :0.01 -> 0.532
3. lr :0.001 -> overfitting->
4. lr:0.01, 1000 estimators->0.539
5. lr : 0.1, no scaling ->0.576
6. lr : 0.01, no scaling ->0.578
7. baseline + count_above_mean ->0.48

### C. Store Indexes

#### Data Download

In [ ]:
X_train_a = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/X_train_1.park")
knowledge = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/knowledge.park")
y_train_a = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/y_train_1.park")

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
df = reduce_mem_usage(X_train_a, verbose=True)

Mem. usage decreased to 555.58 Mb (54.1% reduction)


In [ ]:
df.shape

(1300365, 121)

In [ ]:

#X_train_a["ticker"] = knowledge["ticker"]
#X_train_a["date"] = knowledge["date"]
#X_train_a = X_train_a.sort_values(by=['date'])
#y_train_a = y_train_a.reset_index(drop=True)
#X_train_a = X_train_a.drop(['date'],axis=1)
X_train_a = X_train_a.select_dtypes(include=['float64','int64'])


#X_train_a["ticker"] = knowledge["ticker"]
features = FeatureWiz(corr_limit=0.60, feature_engg='', category_encoders='',dask_xgboost_flag=False, nrows=None, verbose=0)
X_train_a = features.fit_transform(df,y_train_a)



wiz = FeatureWiz(verbose=1)
        X_train_selected = wiz.fit_transform(X_train, y_train)
        X_test_selected = wiz.transform(X_test)
        wiz.features  ### provides a list of selected features ###            
        
############################################################################################
############       F A S T   F E A T U R E  E N G G    A N D    S E L E C T I O N ! ########
# Be judicious with featurewiz. Don't use it to create too many un-interpretable features! #
############################################################################################
correlation limit = 0.6
Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
#### Single_Label Binary_Classification problem ####
    Caution: We will try to reduce the memory usage of dataframe from 565.50 MB
        memory usage after optimization is: 558.06 MB
        decreased by 1.3%
    Loaded train data. Shape

In [ ]:
X_train_a.shape

(1300365, 46)

In [ ]:
del df

In [ ]:
X_train_a["ticker"] = knowledge["ticker"]

#### Processing

In [ ]:
y_train_a = y_train_a.reset_index(drop=True)
#X_train_a['target'] = y_train_a['target']
X_train_a = X_train_a.reset_index(drop=True)

knowledge = knowledge.reset_index(drop=True)
X_train_a['date'] = knowledge['date']
X_train_a['ticker'] = knowledge['ticker']


In [ ]:
X_train_a['target'] = y_train_a['target']

In [ ]:
X_train_a = X_train_a.reset_index(drop=True)

In [ ]:
#X_train_a = X_train_a.dropna(axis=0)
#X_train_a = X_train_a.reset_index(drop=True)

In [ ]:
X_train_a.shape

(1300365, 49)

In [ ]:
def time_series_split(data, split_percent = 0.7):
  min_date = data.date.min()
  max_date = data.date.max()
  #print("Min:", min_date, "Max:", max_date)
  time_between = max_date - min_date
  train_cutoff = min_date + split_percent*time_between
  #print("train_cutoff:", train_cutoff)
  train_df = data[data.date <= train_cutoff]['formulae_index'].tolist()
  test_df = data[data.date > train_cutoff]['formulae_index'].index.tolist()
  #print("Train:", train_df.date.min(), train_df.date.max(), train_df.shape)
  #print("Test:", test_df.date.min(), test_df.date.max(), test_df.shape)
  return train_df, test_df

In [ ]:
X_train_a['formulae_index'] = X_train_a.index

In [ ]:
train_final = []
test_final = []
tickers = X_train_a['ticker'].unique().tolist()
for t in tickers:
  df_temp = X_train_a[X_train_a['ticker'] == t]
  train_data, test_data  = time_series_split(df_temp, split_percent = 0.8)
  if ((len(train_data)>0)&(len(test_data)>0)):
    train_final.extend(train_data)
    test_final.extend(test_data)
  del df_temp

In [ ]:
len(train_final) + len(test_final)

1300365

In [ ]:
import pickle

with open("train_final", "wb") as fp:   #Pickling
  pickle.dump(train_final, fp)


In [ ]:

with open("train_final", "rb") as fp:   # Unpickling
  b = pickle.load(fp)

In [ ]:

with open("test_final", "wb") as fp:   #Pickling
  pickle.dump(test_final, fp)

In [ ]:
with open("test_final", "rb") as fp:   # Unpickling
  c = pickle.load(fp)

### D.Faster Alternatives

In [26]:
df = pd.DataFrame()
df['a']=[1,2,3,4,5,6,7,8,9,10]
df['g'] = [1,1,1,1,2,2,2,2,2,2]

In [32]:
X = df.groupby('g').rolling(3).apply(np.mean,engine='numba',raw=True).add_suffix("{}".format('abcd'))

In [33]:
X

aabcd
g         
1 0    NaN
  1    NaN
  2    2.0
  3    3.0
2 4    NaN
  5    NaN
  6    6.0
  7    7.0
  8    8.0
  9    9.0

In [43]:
indexes = [0,4]

In [44]:
remove_indexes = []

In [45]:
window = 3

In [46]:
for item in indexes:
  remove_indexes.extend(range(item,item+window-1))

In [47]:
remove_indexes

[0, 1, 4, 5]

In [35]:
x2 = df['a'].rolling(3).apply(np.mean,engine='numba',raw=True)
x2

0    NaN
1    NaN
2    2.0
3    3.0
4    4.0
5    5.0
6    6.0
7    7.0
8    8.0
9    9.0
Name: a, dtype: float64

In [50]:
x2 = pd.DataFrame(x2)
x2

,a
0,NaN
1,NaN
2,2.0
3,3.0
4,4.0
5,5.0
6,6.0
7,7.0
8,8.0
9,9.0


In [51]:
x3 = x2[~x2.index.isin(remove_indexes)]

In [52]:
x3

,a
2,2.0
3,3.0
6,6.0
7,7.0
8,8.0
9,9.0


In [53]:
df = pd.DataFrame()
df['a']=[1,2,3,4,5,6,7,8,9,10]
df['g'] = [1,1,1,1,2,2,2,2,2,2]
df['b']=[1,2,3,4,5,6,7,8,9,10]

In [54]:
X = df.groupby('g').rolling(3).apply(np.mean,engine='numba',raw=True).add_suffix("{}".format('abcd'))

In [55]:
X

aabcd  babcd
g                
1 0    NaN    NaN
  1    NaN    NaN
  2    2.0    2.0
  3    3.0    3.0
2 4    NaN    NaN
  5    NaN    NaN
  6    6.0    6.0
  7    7.0    7.0
  8    8.0    8.0
  9    9.0    9.0

In [58]:
x2 = df[['a','b']].rolling(3).apply(np.mean,engine='numba',raw=True).add_suffix("{}".format('_abcd'))
x2

,a_abcd,b_abcd
0,NaN,NaN
1,NaN,NaN
2,2.0,2.0
3,3.0,3.0
4,4.0,4.0
5,5.0,5.0
6,6.0,6.0
7,7.0,7.0
8,8.0,8.0
9,9.0,9.0


#### Data Download

In [72]:
X_train_a = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/X_train_1.park")
knowledge = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/knowledge.park")
y_train_a = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/y_train_1.park")

In [73]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [74]:
df = reduce_mem_usage(X_train_a, verbose=True)

Mem. usage decreased to 555.58 Mb (54.1% reduction)


In [75]:
df.shape

(1300365, 121)

In [76]:

#X_train_a["ticker"] = knowledge["ticker"]
#X_train_a["date"] = knowledge["date"]
#X_train_a = X_train_a.sort_values(by=['date'])
#y_train_a = y_train_a.reset_index(drop=True)
#X_train_a = X_train_a.drop(['date'],axis=1)
X_train_a = X_train_a.select_dtypes(include=['float64','int64'])


#X_train_a["ticker"] = knowledge["ticker"]
features = FeatureWiz(corr_limit=0.60, feature_engg='', category_encoders='',dask_xgboost_flag=False, nrows=None, verbose=0)
X_train_a = features.fit_transform(df,y_train_a)



wiz = FeatureWiz(verbose=1)
        X_train_selected = wiz.fit_transform(X_train, y_train)
        X_test_selected = wiz.transform(X_test)
        wiz.features  ### provides a list of selected features ###            
        
############################################################################################
############       F A S T   F E A T U R E  E N G G    A N D    S E L E C T I O N ! ########
# Be judicious with featurewiz. Don't use it to create too many un-interpretable features! #
############################################################################################
correlation limit = 0.6
Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
#### Single_Label Binary_Classification problem ####
    Caution: We will try to reduce the memory usage of dataframe from 565.50 MB
        memory usage after optimization is: 558.06 MB
        decreased by 1.3%
    Loaded train data. Shape

In [77]:
X_train_a.shape

(1300365, 46)

In [78]:
del df

In [79]:
#X_train_a["ticker"] = knowledge["ticker"]

#### Processing

In [80]:
knowledge = knowledge.reset_index(drop=True)

In [81]:
X_train_a.shape

(1300365, 46)

In [84]:
X_train_a = X_train_a.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64'])

In [85]:
X_train_a2.shape

(1300365, 46)

In [86]:
tickers = knowledge['ticker']

In [ ]:
ticker_indexes = []

In [89]:
knowledge['ticker'].unique()

array(['A', 'AAAB', 'AAAP', ..., 'ZYTO', 'ZYXI', 'ZZ'], dtype=object)

In [100]:
idx = knowledge.drop_duplicates('ticker').index.tolist()

In [101]:
idx[0:3]

[0, 218, 269]

In [102]:
window = 4
remove_indexes = []
for item in idx:
  remove_indexes.extend(range(item,item+window-1))

In [131]:
start_time = time.time()
X = X_train_a.rolling(window).apply(np.mean,engine='numba',raw=True).add_suffix("{}".format('mean'))
print("--- %s seconds ---" % (time.time() - start_time))

--- 3.5917162895202637 seconds ---


In [111]:
X.shape

(1300365, 46)

In [112]:
X_train_a2 = X_train_a.copy()

In [118]:
knowledge = knowledge.reset_index(drop=True)

In [124]:
X_train_a2['ticker'] = knowledge['ticker'].tolist()

In [125]:
X_train_a2.shape

(1300365, 47)

In [129]:
import time

In [130]:
start_time = time.time()
X2 = X_train_a2.groupby('ticker').rolling(4).apply(np.mean,engine='numba',raw=True).add_suffix("{}".format('_abcd'))
print("--- %s seconds ---" % (time.time() - start_time))


--- 28.03977060317993 seconds ---


In [132]:
X.shape

(1300365, 46)

In [137]:
X2.shape

(1300365, 46)

In [138]:
X2.head()

currentratio_abcd  deferredrev_abcd  delay_abcd  \
ticker id                                                              
A      A_1999-11-30                NaN               NaN         NaN   
       A_1999-12-31                NaN               NaN         NaN   
       A_2000-01-31                NaN               NaN         NaN   
       A_2000-02-29           2.162109       242500000.0        99.0   
       A_2000-03-31           2.209961       251750000.0        82.0   

                     divyield_abcd  dps_abcd  evebit_abcd  evebitda_abcd  \
ticker id                                                                  
A      A_1999-11-30            NaN       NaN          NaN            NaN   
       A_1999-12-31            NaN       NaN          NaN            NaN   
       A_2000-01-31            NaN       NaN          NaN            NaN   
       A_2000-02-29            0.0       0.0         39.0      24.243999   
       A_2000-03-31            0.0       0.0         39.0      24.243999   

                     grossmargin_abcd  log_return_7_mean_mean_monthly_abcd  \
ticker id                                                                    
A      A_1999-11-30               NaN                                  NaN   
       A_1999-12-31               NaN                                  NaN   
       A_2000-01-31               NaN                                  NaN   
       A_2000-02-29          0.471924                             0.079773   
       A_2000-03-31          0.474915                             0.079773   

                     log_return_7_mean_std_monthly_abcd  ...     ebt_abcd  \
ticker id                                                ...                
A      A_1999-11-30                                 NaN  ...          NaN   
       A_1999-12-31                                 NaN  ...          NaN   
       A_2000-01-31                                 NaN  ...          NaN   
       A_2000-02-29                            0.079773  ...  224000000.0   
       A_2000-03-31                            0.079773  ...  218500000.0   

                     ev_ratios_abcd  netinccmnusd_abcd  \
ticker id                                                
A      A_1999-11-30             NaN                NaN   
       A_1999-12-31             NaN                NaN   
       A_2000-01-31             NaN                NaN   
       A_2000-02-29    27606.150391        146000000.0   
       A_2000-03-31    36737.072266        142250000.0   

                     log_return_mean_monthly_abcd  cashnequsd_abcd  \
ticker id                                                            
A      A_1999-11-30                           NaN              NaN   
       A_1999-12-31                           NaN              NaN   
       A_2000-01-31                           NaN              NaN   
       A_2000-02-29                      0.376648              0.0   
       A_2000-03-31                      0.225522      342000000.0   

                       ncff_abcd  ncfinv_abcd  sharesbas_abcd  accoci_abcd  \
ticker id                                                                    
A      A_1999-11-30          NaN          NaN             NaN          NaN   
       A_1999-12-31          NaN          NaN             NaN          NaN   
       A_2000-01-31          NaN          NaN             NaN          NaN   
       A_2000-02-29   16000000.0          0.0     448500000.0          0.0   
       A_2000-03-31  310000000.0  -10500000.0     450250000.0   13250000.0   

                     volume_pct_std_monthly_abcd  
ticker id                                         
A      A_1999-11-30                          NaN  
       A_1999-12-31                          NaN  
       A_2000-01-31                          NaN  
       A_2000-02-29                    -0.331101  
       A_2000-03-31                    -0.052176  

[5 rows x 46 columns]

In [144]:
X = X.reset_index(drop=True)

In [145]:

X_new = X[~X.index.isin(remove_indexes)]


In [146]:
X_new.shape

(1266867, 46)

In [147]:
X_new.head()

,currentratiomean,deferredrevmean,delaymean,divyieldmean,dpsmean,evebitmean,evebitdamean,grossmarginmean,log_return_7_mean_mean_monthlymean,log_return_7_mean_std_monthlymean,...,ebtmean,ev_ratiosmean,netinccmnusdmean,log_return_mean_monthlymean,cashnequsdmean,ncffmean,ncfinvmean,sharesbasmean,accocimean,volume_pct_std_monthlymean
3,2.162109,242500000.0,99.00,0.0,0.0,39.0,24.243999,0.471924,0.079773,0.079773,...,224000000.0,27606.150391,146000000.0,0.376648,0.000000e+00,16000000.0,0.0,448500000.0,0.0,-0.331101
4,2.209961,251750000.0,82.00,0.0,0.0,39.0,24.243999,0.474915,0.079773,0.079773,...,218500000.0,36737.072266,142250000.0,0.225522,3.420000e+08,310000000.0,-10500000.0,450250000.0,13250000.0,-0.052176
5,2.257812,261000000.0,65.00,0.0,0.0,39.0,24.243999,0.477905,0.079773,0.079773,...,213000000.0,41504.401367,138500000.0,0.034177,6.840000e+08,604000000.0,-21000000.0,452000000.0,26500000.0,-0.011381
6,2.333984,271000000.0,54.50,0.0,0.0,39.0,24.243999,0.480896,0.079773,0.079773,...,207500000.0,42694.365234,134750000.0,0.026639,1.026000e+09,898000000.0,-31500000.0,452000000.0,39750000.0,0.407232
7,2.385742,293000000.0,43.75,0.0,0.0,39.0,24.243999,0.486145,0.079773,0.079773,...,215500000.0,40914.106934,139750000.0,-0.085685,1.270500e+09,891250000.0,-7750000.0,452067992.0,38750000.0,0.638891


In [150]:
X4 = X2.dropna(axis=0)
X4 = X4.reset_index(drop=True)

In [151]:
X4.shape

(1266867, 46)

In [152]:
X_new = X_new.reset_index(drop=True)

In [155]:
X4.sample(5,random_state=42)

,currentratio_abcd,deferredrev_abcd,delay_abcd,divyield_abcd,dps_abcd,evebit_abcd,evebitda_abcd,grossmargin_abcd,log_return_7_mean_mean_monthly_abcd,log_return_7_mean_std_monthly_abcd,...,ebt_abcd,ev_ratios_abcd,netinccmnusd_abcd,log_return_mean_monthly_abcd,cashnequsd_abcd,ncff_abcd,ncfinv_abcd,sharesbas_abcd,accoci_abcd,volume_pct_std_monthly_abcd
594484,0.622070,0.0,46.0,0.0,0.0,-2.00,-2.932000,0.140015,-0.040790,-0.040790,...,-4921000.0,41.482890,-14283000.00,0.118652,41000.0,6849000.00,0.0,66738424.00,-380000.0,0.566257
1105239,2.556641,0.0,45.0,0.0,0.0,38.25,12.770250,0.059746,0.006049,0.006049,...,403077.5,36.257811,284585.25,-0.031771,118929.5,2256930.75,0.0,4535244.00,0.0,0.034122
1244914,2.895996,0.0,33.0,0.0,0.0,8.75,6.032750,0.340088,-0.018784,-0.018784,...,9601250.0,175.866472,9307500.00,-0.115288,124566250.0,4170750.00,0.0,95964042.00,598250.0,0.124169
168202,5.027344,0.0,53.0,0.0,0.0,-5.75,-18.442249,0.590820,0.034838,0.034838,...,-2160500.0,54.428518,-1494500.00,0.105684,22483500.0,56000.00,-5611000.0,12475109.75,-28750.0,0.303306
1052987,1.054199,0.0,47.0,0.0,0.0,-9.50,-0.571500,0.533447,-0.106918,-0.106918,...,-671334.5,2.909632,-686329.50,-0.186768,1253234.5,0.00,0.0,60004.00,0.0,4.837614


In [156]:
X_new.sample(5,random_state=42)

,currentratiomean,deferredrevmean,delaymean,divyieldmean,dpsmean,evebitmean,evebitdamean,grossmarginmean,log_return_7_mean_mean_monthlymean,log_return_7_mean_std_monthlymean,...,ebtmean,ev_ratiosmean,netinccmnusdmean,log_return_mean_monthlymean,cashnequsdmean,ncffmean,ncfinvmean,sharesbasmean,accocimean,volume_pct_std_monthlymean
594484,0.622070,0.0,46.0,0.0,0.0,-2.00,-2.932000,0.140015,-0.040790,-0.040790,...,-4921000.0,41.482890,-14283000.00,0.118652,41000.0,6849000.00,0.0,66738424.00,-380000.0,0.566257
1105239,2.556641,0.0,45.0,0.0,0.0,38.25,12.770250,0.059746,0.006049,0.006049,...,403077.5,36.257811,284585.25,-0.031771,118929.5,2256930.75,0.0,4535244.00,0.0,0.034122
1244914,2.895996,0.0,33.0,0.0,0.0,8.75,6.032750,0.340088,-0.018784,-0.018784,...,9601250.0,175.866472,9307500.00,-0.115288,124566250.0,4170750.00,0.0,95964042.00,598250.0,0.124169
168202,5.027344,0.0,53.0,0.0,0.0,-5.75,-18.442249,0.590820,0.034838,0.034838,...,-2160500.0,54.428518,-1494500.00,0.105684,22483500.0,56000.00,-5611000.0,12475109.75,-28750.0,0.303306
1052987,1.054199,0.0,47.0,0.0,0.0,-9.50,-0.571500,0.533447,-0.106918,-0.106918,...,-671334.5,2.909632,-686329.50,-0.186768,1253234.5,0.00,0.0,60004.00,0.0,4.837614
